In [1]:
import os
import re

import pickle
import json

import numpy as np

import pandas as pd

In [2]:
folders = ['01_09','01_21','11_21','12_12','09_02','11_29']

In [3]:
repetitions =[]
for fold in folders:
    repetitions = repetitions+[sum(1 for sub in os.listdir(fold) if sub.startswith('Ripetizione_'))]
repetitions


[5, 4, 6, 6, 3, 5]

In [6]:
for nn,fold in enumerate(folders):
    reps=repetitions[nn]

    data_sep = [None] * reps
    dataRAW_sep = [None] * reps
    data_ent = [None] * reps
    dataRAW_ent = [None] * reps
    for rep in range(reps):
        sepdir = fold + "/Ripetizione_" + str(rep) + "/Separabili/"
        entdir = fold + "/Ripetizione_"+ str(rep) + "/Entangled/"

        data_sep[rep] = []
        dataRAW_sep[rep] = []

        for filename in os.listdir(sepdir):
            if filename.endswith(".pickle"):
                file_path = os.path.join(sepdir, filename)
                df = pd.read_pickle(file_path)
                data_sep[rep].append(df)
                filename = os.path.splitext(filename)[0]
                filename = filename + "_raw.pickle"
                file_path = os.path.join(sepdir+"Raw/", filename)
                num_st = int(re.search(r'Dataset_Quantum_stato_(\d+)_ent_0', filename).group(1))
                df = pd.json_normalize(pd.read_pickle(file_path)['Raw_data'])
                df.insert(0, 'State', num_st)
                dataRAW_sep[rep].append(df)


        data_ent[rep] = []
        dataRAW_ent[rep] = []

        for filename in os.listdir(entdir):
            if filename.endswith(".pickle"):
                file_path = os.path.join(entdir, filename)
                df = pd.read_pickle(file_path)
                data_ent[rep].append(df)
                filename = os.path.splitext(filename)[0]
                filename = filename + "_raw.pickle"
                file_path = os.path.join(entdir+"Raw/", filename)
                num_st = int(re.search(r'Dataset_Quantum_stato_(\d+)_ent_1', filename).group(1))
                df = pd.json_normalize(pd.read_pickle(file_path)['Raw_data'])
                df.insert(0, 'State', num_st)
                dataRAW_ent[rep].append(df)

        data_sep[rep] = pd.concat(data_sep[rep], ignore_index=True)
        dataRAW_sep[rep] = pd.concat(dataRAW_sep[rep], ignore_index=True)
        data_ent[rep] = pd.concat(data_ent[rep], ignore_index=True)
        dataRAW_ent[rep] = pd.concat(dataRAW_ent[rep], ignore_index=True)
        # export data_sep[rep] and data_ent[rep] to json
        data_sep[rep].to_json("./Separabili_" + str(fold) + "_rep_" + str(rep)+ ".json")
        data_ent[rep].to_json( "./Entangled_" + str(fold) + "_rep_" + str(rep)+ ".json")
        dataRAW_sep[rep].to_json("./SeparabiliRAW_" + str(fold) + "_rep_" + str(rep)+ ".json")
        dataRAW_ent[rep].to_json( "./EntangledRAW_" + str(fold) + "_rep_" + str(rep)+ ".json")
